In [8]:
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification

from sklearn.datasets import make_regression
from skorch import NeuralNetRegressor

import pandas as pd

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

import torch
from torch import nn
import torch.nn.functional as F
import torch.utils.data
import torchvision
from torchvision import transforms
from PIL import Image

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error

In [9]:
cifar_data = torchvision.datasets.CIFAR10('data/cifar-10/', train=True, download=True)
cifar_testing = torchvision.datasets.CIFAR10('data/cifar-10/', train=False, download=True)

Files already downloaded and verified
Files already downloaded and verified


In [10]:
def sample_mat(mat, n, replace=False, return_idxs=False):
    """Sample `n` random rows from mat.
    Returns an array of the rows sampled from the matrix.
    Arguments:
        mat {np.array} -- A matrix to sample rows from.
        n {int} -- The number of rows to sample.
    Keyword Arguments:
        replace {bool} -- Whether or not to sample with replacement. (default: {False})
        return_idxs {bool} -- Whether or not to return the indexes of the selected rows. (default: {False})
    Returns:
        np.array|tuple -- Returns a matrix containing the sampled rows, or if `return_idxs` is set,
            returns a tuple containing the indexes of the selected rows and the sampled matrix.
    """
    col_idxs = np.random.choice(mat.shape[1], n, replace=replace)
    sampled = mat[:, col_idxs]
    if return_idxs:
        return (col_idxs, sampled)
    return sampled

def sample_mat_row(mat, n, replace=False, return_idxs=False):
    """Sample `n` random rows from mat.
    Returns an array of the rows sampled from the matrix.
    Arguments:
        mat {np.array} -- A matrix to sample rows from.
        n {int} -- The number of rows to sample.
    Keyword Arguments:
        replace {bool} -- Whether or not to sample with replacement. (default: {False})
        return_idxs {bool} -- Whether or not to return the indexes of the selected rows. (default: {False})
    Returns:
        np.array|tuple -- Returns a matrix containing the sampled rows, or if `return_idxs` is set,
            returns a tuple containing the indexes of the selected rows and the sampled matrix.
    """
    col_idxs = np.random.choice(mat.shape[0], n, replace=replace)
    sampled = mat[col_idxs, :]
    if return_idxs:
        return (col_idxs, sampled)
    return sampled    

In [11]:
thing = [[3,1,2],
        [4,1,2],
        [5,1,2]]
testarr = np.array(thing)
col_idxs = np.random.choice(testarr.shape[0], 2, replace=False)
testarr[[1,2], :]

array([[4, 1, 2],
       [5, 1, 2]])

In [12]:
# SVD -
# Ours - Uc @ predicted sigma @ Vr (theyre going to be just the regular U and V for smaller matrices)
# Poormans Uc @ (sampled sigma) @ (Vr)

# reconstruction error | X - Xr |
# plot with X as percent sampled from U and V
# plot another with x as percent of sigma sampled (0 everything after a certain percent)

In [13]:
class RegressorModule(nn.Module):
    def __init__(self, num_units=5000, nonlin=F.relu): # why this param list
        super(RegressorModule, self).__init__()
        self.num_units = num_units
        self.nonlin = nonlin

        self.dense0 = nn.Linear(1024, num_units)
        #self.dropout0 = nn.Dropout(0.7)
        self.dense1 = nn.Linear(num_units,750)
        self.dropout = nn.Dropout(0.7)
        self.dense2 = nn.Linear(750, 300)
        self.dropout2 = nn.Dropout(0.7)
        self.dense3 = nn.Linear(300, 100)
        self.output = nn.Linear(100, 32)

    def forward (self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        #X = self.dropout0(X)
        X = self.nonlin(self.dense1(X))
        X = self.dropout(X)
        X = self.nonlin(self.dense2(X))
        X = self.dropout2(X)
        X = self.nonlin(self.dense3(X))
        X = self.nonlin(self.output(X))
        #X = F.softmax(self.output(X), dim=-1)
        return X

net = NeuralNetRegressor(
    RegressorModule, # pytorch module
    #train_split=None,
    criterion = nn.L1Loss, 
    max_epochs=512,
    lr=0.0001,
    device='cuda',
)

In [14]:
net.initialize()
net.load_params(f_params='models/singular_trained_nodrop.pkl')

FileNotFoundError: [Errno 2] No such file or directory: 'singular_trained.pkl'

X_mats = []
Xr_mats = []
for i in range(1000, 6000, 1000):
    m = i
    n = i

    X = np.random.normal(size=(m, n), scale=10)
    mat_rows = sample_mat_row(X, int(0.2*m))
    mat_cols = sample_mat(X, int(0.2*m))

    # SVD
    U, S, V = np.linalg.svd(X)
    Ur, Sr, Vr = np.linalg.svd(mat_rows)
    Uc, Sc, Vc = np.linalg.svd(mat_cols)

    for j in range(2, 32, 2): 
        S_s = S[0:j]  # zero out the rest
        Xr_s = (U @ np.diag(S_s) @ V)
        Xr = (U @ np.diag(S) @ V)

    # SVD -
    # Ours - Uc @ predicted sigma @ Vr
    # Poormans Uc @ (sampled sigma) @ (Vr)

    # reconstruction error | X - Xr |
    # plot with X as percent sampled from U and V
    # 10 times and make error bar for each percent error (lineplot errorbar)
    # plot another with x as percent of sigma sampled (0 everything after a certain percent)

    X_mats.append(X)
    Xr_mats.append(Xr_s)
    print(f'MSE for {m}x{n}: {mean_squared_error(Xr, X)}')
    print(f'MAE for {m}x{n}: {mean_absolute_error(Xr, X)}')
    print('-----------------------------------------------')


## Poor poor mans vs Real and Our svd vs Real

In [ ]:
real_mats = []
poor_mats = []
our_mats = []
random_mats = []

for i in range(10000):
    img, label = cifar_testing[i]
    g_img = img.convert('LA')
    x = list(g_img.getdata(band=0))

    """REAL"""
    matrix = np.array(x, float)
    matrix.shape = (g_img.size[1], g_img.size[0])
    #matrix = np.matrix(matrix)

    # svd and rank of image
    U, sigma, V = np.linalg.svd(matrix) # computes the SVD
    real_mats.append(matrix)


    """POOOOOR"""
    mat_rows = sample_mat_row(matrix, 15)
    mat_cols = sample_mat(matrix, 15)
    Ur, Sr, Vr = np.linalg.svd(mat_rows)
    Uc, Sc, Vc = np.linalg.svd(mat_cols)

    cut_sigma = sigma
    cut_sigma[10:] = 0

    poor = (Uc @ np.diag(cut_sigma) @ Vr)
    poor_mats.append(poor)

    """OURS"""
    flatten = np.reshape(matrix, (1,1024))
    flatten = flatten.astype(np.float32)
    prediction = net.predict(flatten)

    #print(prediction.shape)
    #print(flatten.shape)
    #print(U.shape)
    #print(V.shape)
    #print(prediction)
    ours = (U @ np.diag(prediction.ravel()) @ V)
    our_mats.append(ours)

    """Random"""
    """OURS"""
    """mat = np.zeros((32,32))
    med = []
    for i in range(20):
        # rand
        mean = []
        mat_rows = sample_mat_row(matrix, 15)
        mat_cols = sample_mat(matrix, 15)
        Ur, Sr, Vr = np.linalg.svd(mat_rows)
        Uc, Sc, Vc = np.linalg.svd(mat_cols)
        random = (Uc @ np.diag(prediction.ravel()) @ Vr)
        med.append(random)

    m = np.dstack(med)
    random_mats.append(np.median(m, 2))"""
  

    random = (Uc @ np.diag(prediction.ravel()) @ Vr)
    random_mats.append(random)




: 

In [ ]:
#poor_real = np.array(real_svd)
#poor_fake = np.array(fake_svd)
real = np.array(real_mats)
poor = np.array(poor_mats)
predicted = np.array(our_mats)
rand = np.array(random_mats)

: 

In [ ]:
rand_int = np.random.randint(0, 10000)
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15,20))

"""Use case for using actual image instead of SVD real"""
if True: 
    img, label = cifar_testing[rand_int]
    g_img = img.convert('LA')

axs[0].imshow(g_img, cmap='gray')
axs[0].set_title('Real')

axs[1].imshow(rand[rand_int], cmap='gray')
axs[1].set_title('Sampled')

axs[2].imshow(predicted[rand_int], cmap='gray')
axs[2].set_title('Predicted')

: 

In [ ]:
"""Poor-mans vs Real error"""
PR_erro = []
for i in range(10000):
    PR_erro.append(mean_absolute_error(real[i], poor[i]))
PR_error = np.array(PR_erro)

"""Real vs Predictions"""
OR_erro = []
for i in range(10000):
    OR_erro.append(mean_absolute_error(real[i], predicted[i]))
OR_error = np.array(OR_erro)

RA_erro = []
for i in range(10000):
    RA_erro.append(mean_absolute_error(real[i], rand[i]))
RA_error = np.array(RA_erro)

: 

In [ ]:
# plotting errors

#plt.plot(np.array(PR_error), label="Poor Error")
plt.plot(np.array(OR_error), label="Pred Error")
plt.plot(np.array(RA_error), label="Subset SVD")
plt.legend()
plt.show()

"""AVG ERRORS"""
APR_error = np.mean(PR_error)
AOR_error = np.mean(OR_error)
print("ERRORS")
print(f'POOR-MANS --- MEAN: {APR_error} \u00B1 {np.std(PR_error)}')
print(f'PREDICTED --- MEAN: {AOR_error} \u00B1 {np.std(OR_error)}')

: 

In [ ]:
APR_error.shape

: 

: 

## Poor poor mans svd. Error vs % of Rols/cols used

In [ ]:
# poor poor mans svd, testing to see how Uc, Vr % affects accuracy
percents_errors = []
min_max = []
for i in range(10000):
    img, label = cifar_testing[i]
    g_img = img.convert('LA')
    x = list(g_img.getdata(band=0))

    matrix = np.array(x, float)
    matrix.shape = (g_img.size[1], g_img.size[0])
    #matrix = np.matrix(matrix)

    # svd and rank of image
    U, sigma, V = np.linalg.svd(matrix) # computes the SVD
    

    for j in range(2,32,2):
        mat_rows = sample_mat_row(matrix, j)
        mat_cols = sample_mat(matrix, j)
        Ur, Sr, Vr = np.linalg.svd(mat_rows)
        Uc, Sc, Vc = np.linalg.svd(mat_cols)
        Xr_s = (Uc @ np.diag(sigma) @ Vr)
        Xr = (U @ np.diag(sigma) @ V)
        percents_errors.append(mean_absolute_error(Xr_s, Xr))
        min_max.append([np.max(Xr), np.min(Xr_s)])



: 

In [ ]:
errors = np.array(percents_errors)
minmax = np.array(min_max)

errors = errors.reshape(-1, 15)
errors.shape
errors_means = np.mean(errors, axis=0)

plt.semilogy(np.arange(2,32,2)/32,errors_means)#.set_title('MAE vs. Number of Rows/cols used (counted in 2)')
plt.title('SemilogY Error vs % Rows/Cols')
# x axis is number of rows/cols used out of the original 32. This counts by 2 (15 is 30 rows/cols). How to make x-axis go by percentage of 32

: 

: 

In [ ]:
# only end, theres huge spike that the true X doesnt reach

: 

In [ ]:
# how to train the model for each 1000-1000 - 5000 for sigma (we prob dont have to since we're just testing whether
# poor mans svd works)
# Are the matrices of both poor-mans and regular svd constructed to become the same size? (both same nxn)
# can't get it to save as a 3x1000x1000 and only as a 3xobjects

# can neural nets take in different sizes than they were trained with

: 